# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Anyone-Can-Learn-To-Code-an-LSTM-RNN-in-Python-(Part-1:-RNN)" data-toc-modified-id="Anyone-Can-Learn-To-Code-an-LSTM-RNN-in-Python-(Part-1:-RNN)-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Anyone Can Learn To Code an LSTM-RNN in Python (Part 1: RNN)</a></div><div class="lev2 toc-item"><a href="#Toy-code-File" data-toc-modified-id="Toy-code-File-11"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Toy code File</a></div><div class="lev2 toc-item"><a href="#How-RNN-memorize-in-graphs" data-toc-modified-id="How-RNN-memorize-in-graphs-12"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>How RNN memorize in graphs</a></div><div class="lev3 toc-item"><a href="#h_layer-vs-input_layer-recurrent" data-toc-modified-id="h_layer-vs-input_layer-recurrent-121"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>h_layer vs input_layer recurrent</a></div><div class="lev3 toc-item"><a href="#h_layer-recurrent-structure" data-toc-modified-id="h_layer-recurrent-structure-122"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>h_layer recurrent structure</a></div><div class="lev3 toc-item"><a href="#RNN-forward-and-backward-passes" data-toc-modified-id="RNN-forward-and-backward-passes-123"><span class="toc-item-num">1.2.3&nbsp;&nbsp;</span>RNN forward and backward passes</a></div><div class="lev2 toc-item"><a href="#Diving-into-code" data-toc-modified-id="Diving-into-code-13"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Diving into code</a></div><div class="lev3 toc-item"><a href="#Import" data-toc-modified-id="Import-131"><span class="toc-item-num">1.3.1&nbsp;&nbsp;</span>Import</a></div><div class="lev3 toc-item"><a href="#Sigmoid-and-derivative" data-toc-modified-id="Sigmoid-and-derivative-132"><span class="toc-item-num">1.3.2&nbsp;&nbsp;</span>Sigmoid and derivative</a></div><div class="lev3 toc-item"><a href="#training-dataset-generation" data-toc-modified-id="training-dataset-generation-133"><span class="toc-item-num">1.3.3&nbsp;&nbsp;</span>training dataset generation</a></div><div class="lev3 toc-item"><a href="#Model-parameters-and-initial-weights" data-toc-modified-id="Model-parameters-and-initial-weights-134"><span class="toc-item-num">1.3.4&nbsp;&nbsp;</span>Model parameters and initial weights</a></div><div class="lev3 toc-item"><a href="#Forward-pass" data-toc-modified-id="Forward-pass-135"><span class="toc-item-num">1.3.5&nbsp;&nbsp;</span>Forward pass</a></div><div class="lev3 toc-item"><a href="#Backward-pass" data-toc-modified-id="Backward-pass-136"><span class="toc-item-num">1.3.6&nbsp;&nbsp;</span>Backward pass</a></div><div class="lev3 toc-item"><a href="#Update-weights-and-print-progress" data-toc-modified-id="Update-weights-and-print-progress-137"><span class="toc-item-num">1.3.7&nbsp;&nbsp;</span>Update weights and print progress</a></div>

# Anyone Can Learn To Code an LSTM-RNN in Python (Part 1: RNN)
Baby steps to your neural network's first memories.

- Trask's LSTM post [link](https://iamtrask.github.io/2015/11/15/anyone-can-code-lstm/)     
- How binary addition work [video](https://www.youtube.com/watch?v=ypqYoFbPfTk)      

## Toy code File

In [3]:
%%writefile anyone_can_code_lstm.py

import copy, numpy as np
np.random.seed(0)

# compute sigmoid nonlinearity
def sigmoid(x):
    output = 1/(1+np.exp(-x))
    return output

# convert output of sigmoid function to its derivative
def sigmoid_output_to_derivative(output):
    return output*(1-output)


# training dataset generation
int2binary = {}
binary_dim = 8

largest_number = pow(2,binary_dim)
binary = np.unpackbits(
    np.array([range(largest_number)],dtype=np.uint8).T,axis=1)
for i in range(largest_number):
    int2binary[i] = binary[i]


# input variables
alpha = 0.1
input_dim = 2
hidden_dim = 16
output_dim = 1


# initialize neural network weights
synapse_0 = 2*np.random.random((input_dim,hidden_dim)) - 1
synapse_1 = 2*np.random.random((hidden_dim,output_dim)) - 1
synapse_h = 2*np.random.random((hidden_dim,hidden_dim)) - 1

synapse_0_update = np.zeros_like(synapse_0)
synapse_1_update = np.zeros_like(synapse_1)
synapse_h_update = np.zeros_like(synapse_h)

# training logic
for j in range(10000):
    
    # generate a simple addition problem (a + b = c)
    a_int = np.random.randint(largest_number/2) # int version
    a = int2binary[a_int] # binary encoding

    b_int = np.random.randint(largest_number/2) # int version
    b = int2binary[b_int] # binary encoding

    # true answer
    c_int = a_int + b_int
    c = int2binary[c_int]
    
    # where we'll store our best guess (binary encoded)
    d = np.zeros_like(c)

    overallError = 0
    
    layer_2_deltas = list()
    layer_1_values = list()
    layer_1_values.append(np.zeros(hidden_dim))
    
    # moving along the positions in the binary encoding
    for position in range(binary_dim):
        
        # generate input and output
        X = np.array([[a[binary_dim - position - 1],b[binary_dim - position - 1]]])
        y = np.array([[c[binary_dim - position - 1]]]).T

        # hidden layer (input ~+ prev_hidden)
        layer_1 = sigmoid(np.dot(X,synapse_0) + np.dot(layer_1_values[-1],synapse_h))

        # output layer (new binary representation)
        layer_2 = sigmoid(np.dot(layer_1,synapse_1))

        # did we miss?... if so, by how much?
        layer_2_error = y - layer_2
        layer_2_deltas.append((layer_2_error)*sigmoid_output_to_derivative(layer_2))
        overallError += np.abs(layer_2_error[0])
    
        # decode estimate so we can print it out
        d[binary_dim - position - 1] = np.round(layer_2[0][0])
        
        # store hidden layer so we can use it in the next timestep
        layer_1_values.append(copy.deepcopy(layer_1))
    
    future_layer_1_delta = np.zeros(hidden_dim)
    
    for position in range(binary_dim):
        
        X = np.array([[a[position],b[position]]])
        layer_1 = layer_1_values[-position-1]
        prev_layer_1 = layer_1_values[-position-2]
        
        # error at output layer
        layer_2_delta = layer_2_deltas[-position-1]
        # error at hidden layer
        layer_1_delta = (future_layer_1_delta.dot(synapse_h.T) + layer_2_delta.dot(synapse_1.T)) * sigmoid_output_to_derivative(layer_1)

        # let's update all our weights so we can try again
        synapse_1_update += np.atleast_2d(layer_1).T.dot(layer_2_delta)
        synapse_h_update += np.atleast_2d(prev_layer_1).T.dot(layer_1_delta)
        synapse_0_update += X.T.dot(layer_1_delta)
        
        future_layer_1_delta = layer_1_delta
    

    synapse_0 += synapse_0_update * alpha
    synapse_1 += synapse_1_update * alpha
    synapse_h += synapse_h_update * alpha    

    synapse_0_update *= 0
    synapse_1_update *= 0
    synapse_h_update *= 0
    
    # print out progress
    if(j % 1000 == 0):
        print("Error:",  str(overallError))
        print("Pred:" , str(d))
        print("True:" , str(c))
        out = 0
        for index,x in enumerate(reversed(d)):
            out += x*pow(2,index)
        print(str(a_int), " + " , str(b_int) , " = " , str(out))
        print("------------")

        

Overwriting anyone_can_code_lstm.py


In [5]:
!python anyone_can_code_lstm.py

Error: [ 3.45638663]
Pred: [0 0 0 0 0 0 0 1]
True: [0 1 0 0 0 1 0 1]
9  +  60  =  1
------------
Error: [ 3.63389116]
Pred: [1 1 1 1 1 1 1 1]
True: [0 0 1 1 1 1 1 1]
28  +  35  =  255
------------
Error: [ 3.91366595]
Pred: [0 1 0 0 1 0 0 0]
True: [1 0 1 0 0 0 0 0]
116  +  44  =  72
------------
Error: [ 3.72191702]
Pred: [1 1 0 1 1 1 1 1]
True: [0 1 0 0 1 1 0 1]
4  +  73  =  223
------------
Error: [ 3.5852713]
Pred: [0 0 0 0 1 0 0 0]
True: [0 1 0 1 0 0 1 0]
71  +  11  =  8
------------
Error: [ 2.53352328]
Pred: [1 0 1 0 0 0 1 0]
True: [1 1 0 0 0 0 1 0]
81  +  113  =  162
------------
Error: [ 0.57691441]
Pred: [0 1 0 1 0 0 0 1]
True: [0 1 0 1 0 0 0 1]
81  +  0  =  81
------------
Error: [ 1.42589952]
Pred: [1 0 0 0 0 0 0 1]
True: [1 0 0 0 0 0 0 1]
4  +  125  =  129
------------
Error: [ 0.47477457]
Pred: [0 0 1 1 1 0 0 0]
True: [0 0 1 1 1 0 0 0]
39  +  17  =  56
------------
Error: [ 0.21595037]
Pred: [0 0 0 0 1 1 1 0]
True: [0 0 0 0 1 1 1 0]
11  +  3  =  14
------------


## How RNN memorize in graphs

### h_layer vs input_layer recurrent

In [29]:
from IPython.display import Image

> Hidden recurrence learns what to remember whereas input recurrence is hard wired to just remember the immediately previous datapoint. [go to post](https://hyp.is/gS607gYEEee3C58d8pc57g/iamtrask.github.io/2015/11/15/anyone-can-code-lstm/)

> The network REALLY needs to know more about what part of the song its in. However, the "hidden layer recurrence" doesn't break down in this way. It subtely remembers everything it saw (with memories becoming more subtle as it they fade into the past) [go to post](https://hyp.is/OmTjJgYEEeeIQ_MJ-s_YGQ/iamtrask.github.io/2015/11/15/anyone-can-code-lstm/)

In [30]:
Image(width = 800, height= 500, url='https://lh3.googleusercontent.com/7WNccZ2lMT0YfSkPi1F4dzreHCJ8D7mbKnB6K95hR16Gko7yhbUvp8hGbDW7-N5oVfoi98T0eUYbvdcDEbPsYydMC6B33DFV0b9cubajxHhsyG8AcmpgOx2jMOuiOQ0jXfFSLOUgdaBpu2z9ZRPBFQVnzKcBB80ZmLFjD8IMIpGsOvkT551b4IDKhrM10AyIoPPkoq550_yVZIPPIp6cy_GsI3pOlaxFJrtZTHZeJM8GW05DsL4VpDX1zeMCThdLgPhiSfgCKS6we_WMQqhI_KDvqiR-znFt4rpluIGCoWHeX-W1XLMlWNZVgMFVSOgrSburKNdFaIHKwDTZjZ819_3_gvEMNkCPhHJpBchxgG2-13PZ-YAhWxqPlV_5mMknk79RYt-bfcverJrPnK2O0N2qRbLRXT9Ni4tlRU9eP9DZwnEBeyHkke_BMU8TqslhUt3mehZHW4TkzcaH09kj2phHFbdI38zwPdR0wtrGDSUOUL-cHLmu1tlP6N5ulgxIfVkKVyAnoUkU0RDx1dDhM99JMn2GHxT2q4aYHWgIn3ED4q83QoVLhdlFdteXedaPOEDrpoNyiTQKAPc5PIDC7PBwSlqKTGkRTaeS8DSMIV0c0oD2Tcqa=w1546-h884-no')

### h_layer recurrent structure

- how to calc hidden layer using input and previous hidden layer

In [31]:
Image(width = 500, height=300, url='https://iamtrask.github.io/img/basic_recurrence_singleton.png')

### RNN forward and backward passes

In [32]:
Image(width = 500, height = 300, url='https://iamtrask.github.io/img/recurrence_gif.gif')

In [33]:
Image(width = 500, height = 300, url='https://iamtrask.github.io/img/backprop_through_time.gif')

## Diving into code

### Import

In [2]:
import copy, numpy as np

### Sigmoid and derivative

In [3]:
np.random.seed(0)

# compute sigmoid nonlinearity
def sigmoid(x):
    output = 1/(1+np.exp(-x))
    return output

# convert output of sigmoid function to its derivative
def sigmoid_output_to_derivative(output):
    return output*(1-output)

### training dataset generation

In [4]:
int2binary = {}
binary_dim = 8

largest_number = pow(2,binary_dim)     # 256

In [5]:
# see transformation below
binary = np.unpackbits(
    np.array([range(largest_number)],dtype=np.uint8).T,axis=1)

binary_1 == {{np.array([range(largest_number)],dtype=np.uint8)}}     

binary_2 == {{np.array([range(largest_number)],dtype=np.uint8).T}}       

binary == {{np.unpackbits(np.array([range(largest_number)],dtype=np.uint8).T, axis=1)}}

In [6]:
for i in range(largest_number):
    int2binary[i] = binary[i]

### Model parameters and initial weights

In [7]:
# input variables
alpha = 0.1
                                                        # input layer offer 2 values
input_dim = 2
                                                        # hidden layer output 16 values
hidden_dim = 16
                                                        # output just a single value
output_dim = 1

In [8]:
# initialize neural network weights
synapse_0 = 2*np.random.random((input_dim,hidden_dim)) - 1
synapse_1 = 2*np.random.random((hidden_dim,output_dim)) - 1
synapse_h = 2*np.random.random((hidden_dim,hidden_dim)) - 1

# update to 0s
synapse_0_update = np.zeros_like(synapse_0)
synapse_1_update = np.zeros_like(synapse_1)
synapse_h_update = np.zeros_like(synapse_h)

synapse_0 == {{synapse_0}}

synapse_0_update == {{synapse_0_update}}

### Forward pass

In [16]:
# training logic
                                                                    # num of epochs
for j in range(1):
    
    # generate a simple addition problem (a + b = c)
    
    a_int = np.random.randint(largest_number/2) # int version
    a = int2binary[a_int] # binary encoding
    print("a:", a)
    
    b_int = np.random.randint(largest_number/2) # int version
    b = int2binary[b_int] # binary encoding
    print("b:", b)
    
    # true answer
    c_int = a_int + b_int
    c = int2binary[c_int]
    print("c:", c)
    
    # where we'll store our best guess (binary encoded)
                                                                    # store predictions
    d = np.zeros_like(c)
    print("d:", d)
    
    print("-----------------------------------")

    overallError = 0
    
    layer_2_deltas = list()
    layer_1_values = list()
    
                                                                    # initialize: an element of array_zeros (16,) as first element
                                                                    # hidden_dim = 16
    layer_1_values.append(np.zeros(hidden_dim))
    
    
    
    
    
    # moving along the positions in the binary encoding
                                                                    # full dataset is 8 samples
                                                                    # binary_dim = 8
                                                                    # for 8 digit, one by one moving from right to left
    count=0
    for position in range(binary_dim):      
        

#         print('position:', position)
#         print('--------------')
        
        # generate input and output
                                                                    # X.shape dim(1, 2)
        X = np.array([[a[binary_dim - position - 1],b[binary_dim - position - 1]]])
        y = np.array([[c[binary_dim - position - 1]]]).T
#         print('X:', X)
#         print('y:', y)
        

                                                                    # layer_1_values[-1] always refer to previous layer_1_values    
        # hidden layer (input ~+ prev_hidden)
                             # input_X, W              previous_layer_1, W_h          : to keep memory
        layer_1 = sigmoid(np.dot(X,synapse_0) + np.dot(layer_1_values[-1],synapse_h))
                                                                    # layer_1 return 16 values or 16 neurons
#         print("layer_1", layer_1)
        
        
        
        # output layer (new binary representation)
                                                                    # layer_2 always refer a single value
        layer_2 = sigmoid(np.dot(layer_1,synapse_1))
#         print("layer_2", layer_2)
        
        
                                                                    # not sure what layer_2 is exactly doing
                                                                    # we want network to do binary addtion
                                                                    # backward pass will help us correct network to do addition
                                                                    # iteration by iteration
        # did we miss?... if so, by how much?
        layer_2_error = y - layer_2
                                                                    # store layer_2_delta: gradient?
        layer_2_deltas.append((layer_2_error)*sigmoid_output_to_derivative(layer_2))
#         print('layer_2_deltas:', layer_2_deltas)
        
                                                                    # add up error of a digit out of 8 into overallError
        overallError += np.abs(layer_2_error[0])
#         print("overallError:", overallError)
        
        
        # decode estimate so we can print it out
                                                                    # store prediciton of each digit into d, from right to left
        d[binary_dim - position - 1] = np.round(layer_2[0][0])
#         print('d:', d)
        
        # store hidden layer so we can use it in the next timestep
                                                                    # store hidden layer or layer_1, so that layer_1_values[-1]
                                                                    # can always refer to previous hidden layer's value
        layer_1_values.append(copy.deepcopy(layer_1))
#         print("layer_1_values:", layer_1_values)
  
        if count < 2: 
            print('position:', position)
            print('--------------')
            print('X:', X)
            print('y:', y)
            print("layer_1", layer_1)
            print("layer_2", layer_2)
            print('layer_2_deltas:', layer_2_deltas)
            print("overallError:", overallError)
            print('d:', d)
            print("layer_1_values:", layer_1_values)
        count +=1

a: [0 1 1 1 0 0 0 1]
b: [0 0 0 0 0 0 0 0]
c: [0 1 1 1 0 0 0 1]
d: [0 0 0 0 0 0 0 0]
-----------------------------------
position: 0
--------------
X: [[1 0]]
y: [[1]]
layer_1 [[ 0.52438739  0.6059641   0.55120158  0.52242653  0.46190139  0.57243386
   0.46883406  0.68644385  0.71653238  0.44198326  0.64186088  0.51444344
   0.53396987  0.7008174   0.29777245  0.30454628]]
layer_2 [[ 0.52385887]]
layer_2_deltas: [array([[ 0.11876424]])]
overallError: [ 0.47614113]
d: [0 0 0 0 0 0 0 1]
layer_1_values: [array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.]), array([[ 0.52438739,  0.6059641 ,  0.55120158,  0.52242653,  0.46190139,
         0.57243386,  0.46883406,  0.68644385,  0.71653238,  0.44198326,
         0.64186088,  0.51444344,  0.53396987,  0.7008174 ,  0.29777245,
         0.30454628]])]
position: 1
--------------
X: [[0 0]]
y: [[0]]
layer_1 [[ 0.4607088   0.70122245  0.91939146  0.10792181  0.90421724  0.19833045
   0.15515846  0.49971006 

### Backward pass

In [ ]:
                                                      # set layer_1_delta as 0s of 16
    future_layer_1_delta = np.zeros(hidden_dim)
    

    for position in range(binary_dim):
                                                      # get a and b with the most left digit
        X = np.array([[a[position],b[position]]])
        
                                                      # get the previous layer_1 value
        layer_1 = layer_1_values[-position-1]
                                                      # get the previous previous layer_1 value  
        prev_layer_1 = layer_1_values[-position-2]
        
        # error at output layer
                                                      # get previous layer_2_delta
        layer_2_delta = layer_2_deltas[-position-1]
        
        
        # error at hidden layer
        # layer_1_delta has two parts: 
        #                        from future_layer_1_delta           from layer_2_delta,         
        layer_1_delta = (future_layer_1_delta.dot(synapse_h.T) + layer_2_delta.dot(synapse_1.T)) * sigmoid_output_to_derivative(layer_1)

        
        
        # let's update all our weights 
                                                        # for each of 8 digits in loop
        synapse_1_update += np.atleast_2d(layer_1).T.dot(layer_2_delta)
        synapse_h_update += np.atleast_2d(prev_layer_1).T.dot(layer_1_delta)
        synapse_0_update += X.T.dot(layer_1_delta)
        
        
                                                        # current layer_1_delta becomes future layer_1_delta to next  
        future_layer_1_delta = layer_1_delta
    

### Update weights and print progress

In [ ]:

                                                        # update all weights at end of each epoch using updated part from 
                                                        # the forward and backward above
    
    synapse_0 += synapse_0_update * alpha
    synapse_1 += synapse_1_update * alpha
    synapse_h += synapse_h_update * alpha    

                                                        # get this part of weights back to 0s for next epoch
    synapse_0_update *= 0
    synapse_1_update *= 0
    synapse_h_update *= 0
    
    
                                                        # print out progress or how well this model learned binary addition
                                                        # at end of each 1000 epochs
    # print out progress
    if(j % 1000 == 0):
        print("Error:" + str(overallError))
        print("Pred:" + str(d))
        print("True:" + str(c))
        out = 0
        
                                                        # reversed(d): make sure to loop digit from right to left
        for index,x in enumerate(reversed(d)):
            out += x*pow(2,index)
        print(str(a_int) + " + " + str(b_int) + " = " + str(out))
        print("------------")

        